In [ ]:
from nltk.corpus import stopwords
import statistics
import itertools  
import math
import random
import numpy as np
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
from operator import add,sub
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import os
import matplotlib.pyplot as plt 
import pickle
import re
from natsort import natsorted
import string
from operator import itemgetter
import collections
from num2words import num2words
import math
import numpy as np
import numpy as np
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [ ]:
def readFile(filename):
    file = open(filename, 'r',encoding ="ascii", errors ="surrogateescape")
    document=file.read()
    file.close()
    #print(document)
    #document=remove_header_footer(document)
    #print(document)
    return document
def whitespaceRemoval(document):
    return document.strip()
def stopWordsRemoval(tokens):
    stop_words=set(stopwords.words('english'))
    token_list=[token for token in tokens if not token in stop_words]
    return token_list
def puntuationsRemoval(document):
    table = str.maketrans('', '', string.punctuation)
    document = document.translate(table)
    return document
def tokenization(document):
    token_list=word_tokenize(document)
    return token_list
def lowerCase(tokens):
    token_list=[token.lower() for token in tokens]
    return token_list
def porterStemmer(tokens):
    ps=PorterStemmer()
    token_list=[ps.stem(token) for token in tokens]
    return token_list
def numbersremoval(document):
    document=re.sub(r'\d+','',document)
    return document
def numbersToWords(document):
    temp = re.findall(r'\d+', document) 
    res = list(map(int, temp)) 
    s=""
    for i in res:
        s=s+num2words(i)+" "
    return s;

In [ ]:
#prprocessing steps
def documentToTerms(document):
    token_list=[]
    #print("Number To words Convertion")
    string_of_numbers=numbersToWords(document)
    #print(string_of_numbers)
    document=document+" "+string_of_numbers
    #print(document)
    #Whitespace removal
    #print("Whitespace removal")
    document=whitespaceRemoval(document)
    #Punctuations Removal
    #print("Puntuations removal")
    document=puntuationsRemoval(document)
#     #removing numbers
    document=numbersremoval(document)
#     #print("Removal of Numbers")
    #print(document)
    #tokenization
    #print("Tokenization")
    token_list=tokenization(document)
#     #stop words removal
    #print("Stop Words Removal")
    token_list=stopWordsRemoval(token_list)
    #print(token_list)
    #lower Case conversion
    #print("LowerCase Conversion")
    token_list=lowerCase(token_list)
    #print(token_list)
    #stemming
    #print("Stemming")
    token_list=porterStemmer(token_list)
    #print(token_list)
    return token_list

In [ ]:
#this block of code is meant to keep details of the files and thier folder name and the class id
folders=natsorted(os.listdir("Data/"))
document_details={} #storing details per document id
doc_id=-1
folder_=-1
folder_names={}
for folder in folders:
    fpath='Data/'+ folder
    if(os.path.isdir(fpath)):
        folder_+=1
        folder_names[folder_]=folder
        file_names=natsorted(os.listdir("Data/" + folder))
        for file_name in file_names:
            doc_id+=1
            document_details[doc_id]=[]
            document_details[doc_id].append(folder)
            document_details[doc_id].append(file_name)
            document_details[doc_id].append(folder_)
doc_id+=1
no_folders=len(folder_names)

In [ ]:
def split(ratio): #The splitting function to split the data o train and test by randomize method
    train=random.sample(range(doc_id), int(doc_id*ratio))
    train.sort()
    temp=[i for i in range(doc_id)]
    test=list(set(temp).difference(set(train)))
    test.sort()
    return test,train

In [ ]:
#Buidling inverted index on the basis of class division
def invertedIndex(terms,document_id,class_,index):
    for term in terms:
        if term in index:
            if class_ in index[term][1]:
                if(document_id in index[term][1][class_]):
                    index[term][1][class_][document_id]+=1
                else:
                    index[term][0]+=1
                    index[term][1][class_][document_id]=1
            else:
                index[term][0]+=1
                index[term][1][class_]={}
                index[term][1][class_][document_id]=1
        else:
            index[term]=[]
            index[term].append(1)
            index[term].append({})
            index[term][1][class_]={}
            index[term][1][class_][document_id]=1
    return index

In [ ]:
def indexCreation(train):#Creates the index based only on the vocab of train data
    class_terms={}
    index={}
    for ind in train:
        class_=document_details[ind][2]
        document=readFile("Data/" + document_details[ind][0]+"/"+document_details[ind][1])
        if(class_ in class_terms):
            class_terms[class_]+=1
        else:
            class_terms[class_]=1
        #preprocessing from the document
        terms=list(documentToTerms(document))
        index=invertedIndex(terms,ind,class_,index)
#     print("Index:",index)
#     print("class:",class_terms)
    return index,class_terms

In [ ]:
def topKTFIDF(d,k):#the top K items of the dictionary.Just a helper class
    d={key: value for key, value in sorted(d.items(),reverse=True, key=lambda item: item[1])}
    if len(d)>k:
        d = dict(list(d.items())[0: k])     
    return d

In [ ]:
def tfidfFeatureSelection(features_tf_idf,k,index):# the main alogorithm for TFifd features selection
    for key,value in index.items():
        idf=math.log(no_folders/len(value[1])) #idf= presence of term in how many classes
        for key_,v in value[1].items():
            tf=1+math.log(sum(v.values())) #tf=count of terms in all documnest  of the class
            features_tf_idf[key_][key]=tf*idf
    features_tf_idf=[topKTFIDF(i,k) for i in features_tf_idf]
    return features_tf_idf

In [ ]:
def miFunc(N11,N10,N01,N00):#helper fucnction for evaluatiing MI
    N1_=N10+N11
    N0_=N00+N01
    N_1=N01+N11
    N_0=N10+N00
    N=N11+N01+N10+N00
    
    a=(N11/N)*math.log2((N*N11)/(N1_*N_1)) if ((N1_*N_1)!=0 and (N*N11)/(N1_*N_1)!=0 and N!=0) else 0
    b=(N10/N)*math.log2((N*N10)/(N1_*N_0)) if ((N1_*N_0)!=0 and (N*N10)/(N1_*N_0)!=0 and  N!=0) else 0
    c=(N00/N)*math.log2((N*N00)/(N0_*N_0)) if ((N0_*N_0)!=0 and (N*N00)/(N0_*N_0)!=0 and N!=0) else 0
    d=(N01/N)*math.log2((N*N01)/(N0_*N_1)) if ((N0_*N_1)!=0 and (N*N01)/(N0_*N_1)!=0 and N!=0) else 0
    I=a+b+c+d
    return I

In [ ]:
def miFeatureSelection(features_mi,k,index,class_terms):#main fucntion of MI based features selecton
    for key,value in index.items():
        for key_,v in value[1].items():
            N11=len(v)
            N10=value[0]-N11
            N01=class_terms[key_]-N11
            N00=sum(class_terms.values())-(N11+N01+N10)
            features_mi[key_][key]=miFunc(N11,N10,N01,N00)
    features_mi=[topKTFIDF(i,k) for i in features_mi]
    return features_mi

In [ ]:
def naiveBayesTrain(features,index,class_terms):#Trainaing teh Naive bayes classifier
    prior_classes={}
    prob_term={}
    temp={}
    for i in range(len(features)):
        prior_classes[i]=class_terms[i]/sum(class_terms.values())#stores priors probabability
        prob_term[i]={} #stores the probability of the term for each class
        temp[i]={}
        for key,value in features[i].items():
            prob_term[i][key]=sum(index[key][1][i].values())
        temp[i]=sum(prob_term[i].values())
        prob_term[i]= {k: (v+1)/(temp[i]+len(index)) for k, v in prob_term[i].items()}
    return prob_term,prior_classes,temp

In [ ]:
def naiveBayesTest(prob_term,prior_classes,test,index,class_terms,temp): #fucntion for testing the naive bayes model on test dat
    score=[math.log(prior_classes[i]) for i in range(len(prior_classes))] 
    for term in test:
        if(term in index):
            for key,value in prob_term.items():
                if(term in value):
                    score[key]+=math.log(value[term])
                else:
                    score[key]+=math.log(1/(len(index)+temp[key]))
    predicted=folder_names[score.index(max(score))]
    return predicted

In [ ]:
def lengthDocuments(document): #calaclates the length of each documents and stores in length
    temp=list(document.values())
    temp=[ i*i for i in temp]
    length=math.sqrt(sum(temp))
    return length

In [ ]:
def documentToVector(document_id,class_,features,no_documents): #converting a partuclar docuemnent to vector form
    doc_vector={}
    for key,value in features[class_].items():
        if document_id in index[key][1][class_]:
            tf=1+math.log(index[key][1][class_][document_id])
            idf=math.log(no_documents/index[key][0])
            doc_vector[key]=tf*idf
    return doc_vector

In [ ]:
def cosineSimilairity(query,document): #performs cosine similarity assuming query vector is 1 for term present in query i.e fast cosine
    score=0
    for term in query:
        if term in document:
            score+=document[term]
    length=lengthDocuments(document)
    score=score/length if length!=0 else 0
    return score

In [ ]:
 def knn(train,features,test_vector,kForKNN,index,class_terms):#main fucntion for kNN calaultion
    score={}
    for ind in train:
        doc_vector=documentToVector(ind,document_details[ind][2],features,sum(class_terms.values()))
        score[ind]=cosineSimilairity(test_vector,doc_vector)
    score=topKTFIDF(score,kForKNN)
    class_count=[0 for i in range(no_folders)]
    for key,values in score.items():
        class_count[document_details[key][2]]+=1
    label=folder_names[class_count.index(max(class_count))]#as similarity is taken so maximiu. score is condiderd
    return label

In [ ]:
def metrics(test,train,method,features,kForKNN,index,class_terms):# the fucntion thet evaluates on all test data
    predicted=[]
    actual=[]
    for ind in test:
        doc=readFile("Data/" + document_details[ind][0]+"/"+document_details[ind][1])
        doc=documentToTerms(doc)
        if method=='Naive':
            prob_term,prior_classes,temp=naiveBayesTrain(features,index,class_terms)
            predicted.append(naiveBayesTest(prob_term,prior_classes,doc,index,class_terms,temp))
        elif method=='KNN':
            predicted.append(knn(train,features,doc,kForKNN,index,class_terms))
        actual.append(document_details[ind][0])
    accuarcy=accuracy_score(actual,predicted)
    print(confusion_matrix(actual, predicted,labels=list(folder_names.values())))
    print(accuarcy)
    return accuarcy,confusion_matrix

In [ ]:
def plott(X,plot1,plot2,r):#the plotting fucntion in general
    Xi = list(range(len(X)))
    plt.xticks(Xi, X)
    plt.plot(Xi,plot1,marker='o',label = "MI") 
    plt.plot(Xi,plot2,marker='o',label = "TFIDfF") 
    plt.xlabel('No of features')
    plt.ylabel('accuracy')
    plt.title("Naive: Split_Ratio="+ str(r)) 
    plt.legend()
    plt.show()

In [ ]:
#For Naive at varying features value MI and TF performace at given split
#Simiarly we will do for Knn
ratio=[0.8]
for i in ratio:
    print("Split ratio:",i)
    test,train=split(i)
    index,class_terms=indexCreation(train)
    K=[1,10,100,1000,10000]
    accuarcy_mi=[]
    accuarcy_tfidf=[]
    X=[]
    features_tf_idf=[{} for i in range(no_folders)]
    features_mi=[{} for i in range(no_folders)]
    kForKNN=int(math.sqrt(sum(class_terms.values())))
    for k in K:
        print("No of features:",k)
        features_tf_idf=tfidfFeatureSelection(features_tf_idf,k,index)
        features_mi=miFeatureSelection(features_mi,k,index,class_terms)
        method=['Naive']
        print("MI")
        accuarcy,confusion_matrix=metrics(test,train,method[0],features_mi,kForKNN,index,class_terms)
        accuarcy_mi.append(accuarcy)
        print("TFIDF")
        accuarcy,confusion_matrix=metrics(test,train,method[0],features_tf_idf,kForKNN,index,class_terms)
        accuarcy_tfidf.append(accuarcy)
        X.append(k)
    plott(X,accuarcy_mi,accuarcy_tfidf,ratio[0])
        

In [ ]:
def plot(X,plot1,plot2,r):
    Xi = list(range(len(X)))
    plt.xticks(Xi, X)
    plt.plot(Xi,plot1,marker='o',label = "MI")
    plt.plot(Xi,plot2,marker='o',label = "TFIDF")
    plt.xlabel('Split Ratio')
    plt.ylabel('Accuracy')
    plt.title("Naive: Feature count="+str(r)) 
    plt.legend()
    plt.show()

In [ ]:
#For Naive at varying split ratio value MI and TF performace at given feature count
#Simiarly we will do for Knn
ratio=[0.5,0.7,0.8]
accuarcy_mi=[]
accuarcy_tfidf=[]
X=[]
K=[100]
for i in ratio:
    print("Split ratio:",i)
    test,train=split(i)
    index,class_terms=indexCreation(train)
    #K=[1,10,100,1000,10000]
    features_tf_idf=[{} for i in range(no_folders)]
    features_mi=[{} for i in range(no_folders)]
    kForKNN=int(math.sqrt(sum(class_terms.values())))
    for k in K:
        print("No of features:",k)
        features_tf_idf=tfidfFeatureSelection(features_tf_idf,k,index)
        features_mi=miFeatureSelection(features_mi,k,index,class_terms)
        method=['Naive']
        print("MI")
        accuarcy,confusion_matrix=metrics(test,train,method[0],features_mi,kForKNN,index,class_terms)
        accuarcy_mi.append(accuarcy)
        print("TFIDF")
        accuarcy,confusion_matrix=metrics(test,train,method[0],features_tf_idf,kForKNN,index,class_terms)
        accuarcy_tfidf.append(accuarcy)
    X.append(i)
plot(X,accuarcy_mi,accuarcy_tfidf,K[0])

In [ ]:
def plot1(X,plot1,plot2,k,r):
    Xi = list(range(len(X)))
    plt.xticks(Xi, X)
    plt.plot(Xi,plot1,marker='o',label = "MI") 
    plt.plot(Xi,plot2,marker='o',label = "TFIDF") 
    plt.xlabel('K_for_Knn')
    plt.ylabel('accuracy')
    plt.title('KNN:No of features='+str(k)+" Split_Ratio="+str(r)) 
    plt.legend()
    plt.show()

In [ ]:
#For KNN at varying K value for KNN value MI and TF performace at given split
ratio=[0.8]
for i in ratio:
    print("Split ratio:",i)
    test,train=split(i)
    index,class_terms=indexCreation(train)
    K=[100]
    accuarcy_mi=[]
    accuarcy_tfidf=[]
    X=[]
    features_tf_idf=[{} for i in range(no_folders)]
    features_mi=[{} for i in range(no_folders)]
    kForKNN=[1,3,5,7]
    print("No of features:",K[0])
    features_tf_idf=tfidfFeatureSelection(features_tf_idf,K[0],index)
    features_mi=miFeatureSelection(features_mi,K[0],index,class_terms)
    for m in kForKNN:
        print("K for KNN:",kForKNN)
        method=['KNN']
        print("MI")
        accuarcy,confusion_matrix=metrics(test,train,method[0],features_mi,m,index,class_terms)
        accuarcy_mi.append(accuarcy)
        print("TFIDF")
        accuarcy,confusion_matrix=metrics(test,train,method[0],features_tf_idf,m,index,class_terms)
        accuarcy_tfidf.append(accuarcy)
        X.append(m)
    plot1(X,accuarcy_mi,accuarcy_tfidf,K[0],ratio[0])
        

In [ ]:
def plot3(X,plot1,plot2,r,k):
    Xi = list(range(len(X)))
    plt.xticks(Xi, X)
    plt.plot(Xi,plot1,marker='o',label = "NAIVE")
    plt.plot(Xi,plot2,marker='o',label = "KNN")
    plt.xlabel('No of features')
    plt.ylabel('Accuracy')
    plt.title("Naive V/S KNN for TFIDF: Split_Ratio="+ str(r)+" K_for_KNN="+str(k)) 
    plt.legend()
    plt.show()

In [ ]:
#For Naive V/s KNN for at varying features value for MI at given split
#Similarylu for TFIDF
ratio=[0.8]
for i in ratio:
    print("Split ratio:",i)
    test,train=split(i)
    index,class_terms=indexCreation(train)
    K=[1,10,100,1000,2000]
    accuarcy_naive=[]
    accuarcy_knn=[]
    X=[]
    #features_tf_idf=[{} for i in range(no_folders)]
    features_mi=[{} for i in range(no_folders)]
    kForKNN=5
    for k in K:
        print("No of features:",k)
        print("K for KNN:",kForKNN)
        #features_tf_idf=tfidfFeatureSelection(features_tf_idf,k,index)
        features_mi=miFeatureSelection(features_mi,k,index,class_terms)
        print("Naive")
        accuarcy,confusion_matrix=metrics(test,train,'Naive',features_mi,kForKNN,index,class_terms)
        accuarcy_naive.append(accuarcy)
        print("KNN")
        accuarcy,confusion_matrix=metrics(test,train,'KNN',features_mi,kForKNN,index,class_terms)
        accuarcy_knn.append(accuarcy)
        X.append(k)
        print("KNN:",accuarcy_knn)
        print("KNN:",accuarcy_naive)
    plot3(X,accuarcy_naive,accuarcy_knn,ratio[0],kForKNN)